# Homework: Decision Tree
## Phase 1: Implementing the ID3 Decision Tree Algorithm
***

## Import Libraries

In [1]:
# complete
import numpy as np
import pandas as pd
import csv
import random as rnd

## Load Dataset (CSV file)

In [2]:
# complete
train_data = pd.read_csv('PlayTennis_train.csv')
test_data = pd.read_csv('PlayTennis_test.csv')

In [3]:
num_rows , num_columns = train_data.shape
print(f'(num_rows , num_columns) :{num_rows , num_columns}')
print(f'features : \n{train_data["Outlook"]}\n{train_data["Wind"]}\n{train_data["Humidity"]}\n{train_data["Temperature"]}')
print(f'label :\n{train_data["Play Tennis"]}')
print(f'\nnum of calsses(Yes and NO) :2')

(num_rows , num_columns) :(14, 5)
features : 
0        Sunny
1        Sunny
2     Overcast
3         Rain
4         Rain
5         Rain
6     Overcast
7        Sunny
8        Sunny
9         Rain
10       Sunny
11    Overcast
12    Overcast
13        Rain
Name: Outlook, dtype: object
0       Weak
1     Strong
2       Weak
3       Weak
4       Weak
5     Strong
6     Strong
7       Weak
8       Weak
9       Weak
10    Strong
11    Strong
12      Weak
13    Strong
Name: Wind, dtype: object
0       High
1       High
2       High
3       High
4     Normal
5     Normal
6     Normal
7       High
8     Normal
9     Normal
10    Normal
11      High
12    Normal
13      High
Name: Humidity, dtype: object
0      Hot
1      Hot
2      Hot
3     Mild
4     Cool
5     Cool
6     Cool
7     Mild
8     Cool
9     Mild
10    Mild
11    Mild
12     Hot
13    Mild
Name: Temperature, dtype: object
label :
0      No
1      No
2     Yes
3     Yes
4     Yes
5      No
6     Yes
7      No
8     Yes
9     Yes


## Calculating the entropy of the whole dataset

In [4]:
def calc_total_entropy(train_data :pd.DataFrame, label, class_list):
    # complete
    total_row = train_data.shape[0] #the total size of the dataset (training samples)
    total_entr = 0
    
    for c in class_list: 
        class_count = train_data[train_data[label] == c].shape[0] #number of samples for each output class attribute
        p = class_count/total_row
        if (p == 0) : 
            total_entr += 0
        else :
            total_entr += (-p)*np.log2(p) #adding to the total entropy
    
    return total_entr

## Calculating the entropy for a feature

In [5]:
def calc_entropy(data :pd.DataFrame, feature, feature_value, label, class_list):
    # complete
    feature_value_data = data[data[feature] == feature_value]   # finding samples that their choosen feature has the choosen feature_value
    class_count = feature_value_data.shape[0]    
    entropy = 0
    
    for c in class_list:
        label_class_count = feature_value_data[feature_value_data[label] == c].shape[0] # how many of each output class attribute in the sliced datas
        if label_class_count != 0:
            entropy += (-label_class_count/class_count)* np.log2(label_class_count/class_count) # calculating entropy for each output class attribute
    return entropy

## Calculating information gain for a feature

In [6]:
def calc_info_gain(feature_name, data :pd.DataFrame, label, class_list):
    # complete
    feature_value_list = data[feature_name].unique() #unqiue values of the feature (finding all the values of the feature)
    total_row = data.shape[0]
    feature_info = 0.0
    
    for feature_value in feature_value_list:
        feature_value_data = data[data[feature_name] == feature_value] #filtering rows with that feature_value
        feature_value_count = feature_value_data.shape[0]   # number of datas with the desired feature_value
        feature_entropy = calc_entropy(data,feature_name,feature_value, label, class_list) #calculcating entropy for the feature value
        feature_value_probability = feature_value_count/total_row
        feature_info += feature_value_probability * feature_entropy #calculating IG of the feature
        
    return calc_total_entropy(data, label, class_list) - feature_info #calculating information gain of the feature by subtracting

## Finding the most informative feature (feature with highest information gain)

In [7]:
def find_most_informative_feature(train_data :pd.DataFrame, label, class_list):
    # complete
    feature_list = train_data.columns.drop(label) #finding the feature names in the dataset, label is not a feature, so dropping it
    max_info_gain = -1
    max_info_feature = None
    
    for feature in feature_list:  #for each feature in the dataset
        feature_info_gain = calc_info_gain(feature, train_data, label, class_list)
        if max_info_gain < feature_info_gain: #selecting feature name with highest information gain
            max_info_gain = feature_info_gain
            max_info_feature = feature
            
    return max_info_feature

## Adding a node to the tree

In [8]:
def generate_sub_tree(feature_name, train_data :pd.DataFrame, label, class_list):
    tree = {} #sub tree or node
    feature_value_count_dict = train_data[feature_name].value_counts(sort=False) #dictionary of the count of unqiue feature value
    
    for feature_value, count in feature_value_count_dict.items():
        feature_value_data = train_data[train_data[feature_name] == feature_value] #dataset with only feature_name = feature_value
        
        assigned_to_node = False #flag for tracking feature_value is pure class or not
        for c in class_list: #for each class
            class_count = feature_value_data[feature_value_data[label] == c].shape[0] #count of class c

            if class_count == count: #count of (feature_value = count) of class (pure class)
                assigned_to_node = True
                tree[feature_value] = c     #adding node to the tree
                train_data = train_data[train_data[feature_name] != feature_value]  #removing rows with feature_value
        if not assigned_to_node: #not pure class
            tree[feature_value] = "?" #as feature_value is not a pure class, it should be expanded further, so the branch is marking with ?
    return tree, train_data

##  Performing ID3 Algorithm and generating Tree

In [9]:
def make_tree(root, prev_feature, prev_feature_value, train_data, label, class_list, depth):
    # complete
    
    depth-=1
    if depth == -1:
        for nodes, branch in root.items(): #iterating the tree node
            if branch == "?": #if it is expandable
                feature_value_data = train_data[train_data[prev_feature] == nodes] #using the updated dataset
                max_count = 0
                final_label = None
                rnd.shuffle(class_list) # randomely ordering class lables
                for c in class_list:
                    class_count = feature_value_data[feature_value_data[label] == c].shape[0] #count of label c
                    if class_count > max_count:     #finding the label which is repeated the most (assign it to final label of the branch) 
                        max_count = class_count
                        final_label = c
                    if final_label != None : 
                        root[nodes] = final_label                    
        return root

    elif train_data.shape[0] != 0: #if dataset becomes empty after updating
        max_info_feature = find_most_informative_feature(train_data, label, class_list) #most informative feature
        tree, train_data = generate_sub_tree(max_info_feature, train_data, label, class_list) #getting tree node and updated dataset
        next_root = None
        
        if prev_feature_value != None: #add to the previous node of the tree
            root[prev_feature_value] = dict()
            root[prev_feature_value][max_info_feature] = tree
            next_root = root[prev_feature_value][max_info_feature]
        else: #add to root of the tree
            root[max_info_feature] = tree
            next_root = root[max_info_feature]    # initializing the next subtree

        for nodes, branch in list(next_root.items()): #iterating the tree node
            if branch == "?": #if it is expandable
                feature_value_data = train_data[train_data[max_info_feature] == nodes] #using the updated dataset
                make_tree(next_root, max_info_feature, nodes, feature_value_data, label, class_list, depth) #recursive call with updated dataset

## Finding unique classes of the label and Starting the algorithm

In [10]:
def id3(train_data_m :pd.DataFrame, label, depth = None):
    train_data = train_data_m.copy() #getting a copy of the dataset (not deleting rows from the actual dataset)
    tree = {} #tree which will be updated
    class_list = train_data[label].unique() #getting unqiue classes of the label
    if depth == None:
        depth = class_list.shape[0] + 1
    make_tree(tree, None, None, train_data, label, class_list, depth) #start calling recursion
    return tree

## Predicting from the tree

In [11]:
def predict(tree, instance):
    if not isinstance(tree, dict): #if it is leaf node
        return tree #return the value
    else:
        root_node = next(iter(tree)) #getting first key/feature name of the dictionary
        feature_value = instance[root_node] #value of the feature
        if feature_value in tree[root_node]: #checking the feature value in current tree node
            return predict(tree[root_node][feature_value], instance) #goto next feature
        else:
            return None

## Evaluating test dataset

In [12]:
def evaluate(tree, test_data_m, label):
    correct_preditct = 0
    wrong_preditct = 0
    for index, row in test_data_m.iterrows():
        result = predict(tree, test_data_m.iloc[index])
        if result == test_data_m[label].iloc[index]:
            correct_preditct += 1
        else:
            wrong_preditct += 1
    accuracy = correct_preditct / (correct_preditct + wrong_preditct)
    return accuracy

In [13]:
tree = id3(train_data, 'Play Tennis')
print(tree)

{'Outlook': {'Sunny': {'Humidity': {'High': 'No', 'Normal': 'Yes'}}, 'Overcast': 'Yes', 'Rain': {'Wind': {'Weak': 'Yes', 'Strong': 'No'}}}}


In [14]:
accuracy = evaluate(tree, test_data, 'Play Tennis')
print("accuracy:", accuracy)

accuracy: 1.0
